In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2

In [59]:
df = pd.read_csv('train_data_twitter.csv')

In [60]:
df.head()

,text,label
0,i feel sorry about you because your point of v...,0
1,i feel like he s watching quietly because he s...,0
2,i feel im ugly ugly ugly and ugly now,0
3,i feel unwelcome when we can t go inside right...,0
4,i feel an aching need to prove that i can be,0


In [61]:
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    return (" ".join(words)).strip()
testing = df.text[:10]
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))
test_result

['feel sorry point view indoctrinated left wing european historical revisionists textbooks',
 'feel like watching quietly resigned come realize people going die nothing',
 'feel im ugly ugly ugly ugly',
 'feel unwelcome go inside right away hurts feelings',
 'feel aching need prove',
 'usually stop starts feeling awful',
 'overwhelmed experiences people encountered selfishly dont want try explain feel words inadequate capture convey accurately pilgrimage',
 'feel keep getting hurt',
 'dislike seeing friends sad know inevitable amp amp yes get sad u feel hopeless knowing theyre sad n dont show act like everythings fine deep deeeeep way deep theyre feeling rly shitty',
 'ive feeling melancholy adrift disconnected unfulfilled without really understanding']

In [62]:
nums = [0,77832]
print("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%1000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[1] ))                                                                    
    clean_tweet_texts.append(tweet_cleaner(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 1000 of 77832 has been processed
Tweets 2000 of 77832 has been processed
Tweets 3000 of 77832 has been processed
Tweets 4000 of 77832 has been processed
Tweets 5000 of 77832 has been processed
Tweets 6000 of 77832 has been processed
Tweets 7000 of 77832 has been processed
Tweets 8000 of 77832 has been processed
Tweets 9000 of 77832 has been processed
Tweets 10000 of 77832 has been processed
Tweets 11000 of 77832 has been processed
Tweets 12000 of 77832 has been processed
Tweets 13000 of 77832 has been processed
Tweets 14000 of 77832 has been processed
Tweets 15000 of 77832 has been processed
Tweets 16000 of 77832 has been processed
Tweets 17000 of 77832 has been processed
Tweets 18000 of 77832 has been processed
Tweets 19000 of 77832 has been processed
Tweets 20000 of 77832 has been processed
Tweets 21000 of 77832 has been processed
Tweets 22000 of 77832 has been processed
Tweets 23000 of 77832 has been processed
Tweets 24000 of 77832 has been

In [63]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['label'] = df.label
clean_df.head()

,text,label
0,feel sorry point view indoctrinated left wing ...,0
1,feel like watching quietly resigned come reali...,0
2,feel im ugly ugly ugly ugly,0
3,feel unwelcome go inside right away hurts feel...,0
4,feel aching need prove,0


In [64]:
clean_df.to_csv('train_data_twitter1.csv',encoding='utf-8')
#df = pd.read_csv('dataset.csv', encoding='latin-1', header=None, names=cols)
df = pd.read_csv('train_data_twitter1.csv')

In [65]:
df.head()

,Unnamed: 0,text,label
0,0,feel sorry point view indoctrinated left wing ...,0
1,1,feel like watching quietly resigned come reali...,0
2,2,feel im ugly ugly ugly ugly,0
3,3,feel unwelcome go inside right away hurts feel...,0
4,4,feel aching need prove,0


In [66]:
import pandas as pd

# Assuming 'your_file.csv' is your CSV file with unnamed columns
df = pd.read_csv('train_data_twitter1.csv')

# Display the original DataFrame
print("Original DataFrame:")
print(df)

# Find and drop unnamed columns
unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
df = df.drop(columns=unnamed_columns, errors='ignore')

# Display the DataFrame after dropping unnamed columns
print("\nDataFrame after dropping unnamed columns:")
print(df)

# Now you can save the modified DataFrame back to a CSV file if needed
# df.to_csv('modified_file.csv', index=False)


Original DataFrame:
       Unnamed: 0                                               text  label
0               0  feel sorry point view indoctrinated left wing ...      0
1               1  feel like watching quietly resigned come reali...      0
2               2                        feel im ugly ugly ugly ugly      0
3               3  feel unwelcome go inside right away hurts feel...      0
4               4                             feel aching need prove      0
...           ...                                                ...    ...
77827       77827  actively participating marblehead festival art...      5
77828       77828                  feel amazing posted href http www      5
77829       77829               gotta feeling oprah shocked thrilled      5
77830       77830  feel amazed watch photos footage truly unique ...      5
77831       77831  ask feeling like im interviewing im curious ye...      5

[77832 rows x 3 columns]

DataFrame after dropping unnamed columns:

In [70]:
import pandas as pd

# Assuming 'your_file.csv' is your CSV file with unnamed columns
input_file = 'test_data_twitter1.csv'
output_file = 'test_data_twitter2.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Display the original DataFrame
print("Original DataFrame:")
print(df)

# Find and drop unnamed columns
unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
df = df.drop(columns=unnamed_columns, errors='ignore')

# Display the DataFrame after dropping unnamed columns
print("\nDataFrame after dropping unnamed columns:")
print(df)

# Save the modified DataFrame to a new CSV file
df.to_csv(output_file, index=False)

print(f"\nModified DataFrame saved to {output_file}")


Original DataFrame:
       Unnamed: 0                                               text  label
0               0                              feel absolutely broke      0
1               1                            feel hopeless sometimes      0
2               2                             feel heartbroken right      0
3               3            cant feel anything numb eating anything      0
4               4                    always feel unloved unimportant      0
...           ...                                                ...    ...
11995       11995                               still feel bit dazed      5
11996       11996           feel amazed close best scholars thinking      5
11997       11997                 havent feeling overly funny lately      5
11998       11998  went usf tie loose ends feeling pretty impress...      5
11999       11999  feel like anyone else said henry suppose shock...      5

[12000 rows x 3 columns]

DataFrame after dropping unnamed columns:

In [71]:
df = pd.read_csv('test_data_twitter2.csv')

In [72]:
df.head()

,text,label
0,feel absolutely broke,0
1,feel hopeless sometimes,0
2,feel heartbroken right,0
3,cant feel anything numb eating anything,0
4,always feel unloved unimportant,0
